# Topic Modeling - Actividad

Topic modeling es una técnica de aprendizaje automático no supervisado donde intentados descubrir tópicos que son abstractos al texto pero que pueden describir una colección de documentos. Es importante marcar que estos "tópicos" no son necesariamente equivalentes a la interpretación coloquial de tópicos, sino que responden a un patrón que emerge de las palabras que están en los documentos.

La suposición básica para Topic Modeling es que cada documento está representado por una mescla de tópicos, y cada tópico consite en una conlección de palabras.

## Direcciones
Intentaremos construir un pipeline de machine learning donde como entrada recibamos texto, ejecutemos todos los pasos que vimos en este notebook incluyendo:
 - Eliminación de stopwords
 - Tokenización
 - Stemming y Lemmatization
 - Procesamiento especico del tema
 - Creación de features utilizando algun metodo de reducción de dimensionalidad, SVD, LSI, LDA

, para luego utilizar estas features para entrenar un modelo que nos permita predecir alguna propiedad interesante del set de datos. En este caso en particular, donde estamos viendo tweets, algunos casos interesantes podrían ser:
 - Predecir el sector al que pertenece el tweet: Alimentación, Bebidas, etc.
 - Predecir el paso en el Marketing Funel al que pertece
 
En esta actividad les propongo realizar cambios en alguna de las etapas del procesamiento para modificar la performance del modelo resultante

<img src='https://github.com/santiagxf/M72109/blob/master/NLP/Docs/atap_0406.png?raw=1' />

## Modelado

Descargamos el set de datos y lo cargmamos en un DataFrame

In [1]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv --directory-prefix ./Datasets/mascorpus/

--2020-09-01 13:05:20--  https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 512573 (501K) [text/plain]
Saving to: ‘./Datasets/mascorpus/tweets_marketing.csv’

tweets_marketing.cs 100%[===================>] 500.56K  --.-KB/s    in 0.03s   

2020-09-01 13:05:20 (16.4 MB/s) - ‘./Datasets/mascorpus/tweets_marketing.csv’ saved [512573/512573]



In [2]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

### Instalamos las librerias necesarias

In [3]:
!pip install unidecode
!python -m spacy download es_core_news_sm

     |████████████████████████████████| 245kB 5.5MB/s 
     |████████████████████████████████| 16.2MB 102.8MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp36-none-any.whl size=16172934 sha256=b9a8623aeb6d46d08dc215ceb11222b934b6596fba9ec0614d8786da694aa111
  Stored in directory: /tmp/pip-ephem-wheel-cache-_g_u4muj/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


### Construccion de los pasos

**Paso 1:** Instanciamos nuestro preprocesamiento de texto

In [20]:
import unidecode
import spacy
import es_core_news_sm as spa
import re
import sklearn
import nltk
from nltk import stem
from nltk.corpus import stopwords
from nltk.tokenize.casual import TweetTokenizer

class TextNormalizer(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def __init__(self, language='spanish'):
        nltk.download('stopwords')

        self.parser = spa.load() # Cargamos el parser en español
        self.tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True) # Creamos un tokenizer
        self.stemmer = stem.SnowballStemmer(language=language) # Creamos un steammer
        self.lemmatizer = lambda word : " ".join([token.lemma_ for token in self.parser(word)]) # Creamos un lemmatizer
        self.stopwords = set(stopwords.words(language)) # Instanciamos las stopwords en español
        self.urls_regex = re.compile('http\S+') # Usamos una expresion regular para encontrar las URLs
    
    def process_text(self, text):
        tokens = self.tokenizer.tokenize(text)
        tokens = [token for token in tokens if not re.match(self.urls_regex, token)]
        tokens = [token for token in tokens if len(token) > 4]
        tokens = [token for token in tokens if token not in self.stopwords]
        tokens = [unidecode.unidecode(token) for token in tokens] # Quitamos acentos
        tokens = [self.lemmatizer(token) for token in tokens]
        return tokens
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for doc in X:
          yield ' '.join(self.process_text(text=doc))

In [21]:
normalizer = TextNormalizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Paso 2:** Instanciamos nuestro vectorizador, en este caso usando el método TF-IDF

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True, sublinear_tf=True, norm='l2')

**Paso 3:** Instanciamos nuestro generador de features

In [23]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

featurizer = LatentDirichletAllocation(n_components=7)

**Paso 4:** Instanciamos nuestro clasificador que utilizará las features generadas hasta este momento

In [24]:
from sklearn.linear_model import LogisticRegression

estimator = LogisticRegression(max_iter=10000, multi_class='multinomial')

Ensamblamos el pipeline

In [25]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[('normalizer', normalizer), 
                           ('vectorizer', vectorizer),
                           ('featurizer', featurizer),
                           ('estimator', estimator)])

### Evaluación

**Evaluación:** Entrenamos el modelo y testeamos su performance

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], test_size=0.33, stratify=tweets['SECTOR'])

In [27]:
model = pipeline.fit(X=X_train, y=y_train)

In [28]:
predictions = model.predict(X_test)

In [30]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

ALIMENTACION       0.00      0.00      0.00       110
  AUTOMOCION       0.00      0.00      0.00       148
       BANCA       0.00      0.00      0.00       198
     BEBIDAS       0.28      0.27      0.27       223
    DEPORTES       0.30      0.53      0.39       216
      RETAIL       0.24      0.58      0.34       268
       TELCO       0.00      0.00      0.00        79

    accuracy                           0.27      1242
   macro avg       0.12      0.20      0.14      1242
weighted avg       0.15      0.27      0.19      1242



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
